In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
main_table = pd.read_csv("MainTable.csv")
subjects_table = pd.read_csv("Subject.csv")

In [ ]:
main_table.columns

In [ ]:
main_table.head(50)

In [ ]:
subjects_table.head(5)

In [ ]:

problem_counts = main_table.groupby(['SubjectID', 'ProblemID']).size().reset_index(name='problem_count')

average_problems = problem_counts.groupby('SubjectID')['problem_count'].mean().reset_index(name='average_problems')
result_df = pd.merge(subjects_table, average_problems, on='SubjectID', how='left')
result_df = result_df[result_df["X-Grade"] != 0]
result_df.head(5)


In [ ]:
plt.scatter(result_df["average_problems"], result_df["X-Grade"])

plt.xlabel("Average # of Problem Attempts")
plt.ylabel("Grade")
plt.title("Scatter Plot of Grade vs Average # of Problem Attempts")

plt.show()

In [ ]:

# Count occurrences of "Error" in Compile.Result for each SubjectID
error_counts = main_table[main_table["EventType"] == "Compile.Error"].groupby("SubjectID").size().reset_index(name="error_count")

# Compute the average number of errors per SubjectID
average_errors = error_counts.groupby("SubjectID")["error_count"].mean().reset_index(name="average_errors")

# Merge with subjects_table to retain SubjectID and grades
result_df = pd.merge(subjects_table, average_errors, on="SubjectID", how="left")

# Drop rows where 'X-Grade' is 0
result_df = result_df[result_df["X-Grade"] != 0]

# Scatter plot
plt.scatter(result_df["average_errors"], result_df["X-Grade"])

plt.xlabel("Average # of Compilation Errors")
plt.ylabel("Grade")
plt.title("Scatter Plot of Grade vs Average # of Compilation Errors")

plt.show()

In [ ]:
# Step 1: Count the number of attempts per problem
attempts_per_problem = main_table.groupby("ProblemID").size().reset_index(name="total_attempts")

# Step 2: Count the number of unique students who attempted each problem
unique_students_per_problem = main_table.groupby("ProblemID")["SubjectID"].nunique().reset_index(name="num_students")

# Step 3: Calculate the average number of attempts per student for each problem
average_attempts_per_problem = pd.merge(attempts_per_problem, unique_students_per_problem, on="ProblemID")
average_attempts_per_problem["avg_attempts_per_question"] = average_attempts_per_problem["total_attempts"] / average_attempts_per_problem["num_students"]

# Step 4: Plot the average attempts per problem
plt.figure(figsize=(12, 5))
plt.bar(average_attempts_per_problem["ProblemID"].astype(str), average_attempts_per_problem["avg_attempts_per_question"], color='lightcoral', alpha=0.7)

# Step 5: Add labels and title
plt.xlabel("Problem ID")
plt.ylabel("Average Number of Attempts")
plt.title("Average Number of Attempts per Question")
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

# Step 6: Show the plot
plt.show()


In [ ]:
import pandas as pd

# Step 1: Group the data by SubjectID and ProblemID to count the number of attempts
attempts_per_student_problem = main_table.groupby(['SubjectID', 'ProblemID']).size().reset_index(name='attempts')

# Step 2: Pivot the data to have SubjectID as rows and ProblemID as columns
attempts_per_student_problem_pivot = attempts_per_student_problem.pivot(index='SubjectID', columns='ProblemID', values='attempts').fillna(0)

# Step 3: Display the DataFrame with the number of attempts per student per problem
attempts_per_student_problem_pivot.head()


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Step 1: Merge the attempts dataset with subjects_table on SubjectID
merged_df = pd.merge(subjects_table, attempts_per_student_problem_pivot, on="SubjectID", how="left")
merged_df= merged_df[merged_df["X-Grade"] != 0]

# Step 2: Normalize the data (only the problem attempts)
scaler = StandardScaler()
normalized_data = scaler.fit_transform(merged_df.drop(columns=["SubjectID", "X-Grade"]))  # Drop non-relevant columns

# Step 3: Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
merged_df['Cluster'] = kmeans.fit_predict(normalized_data)

# Step 4: Calculate the average X-Grade for each cluster
cluster_avg_grades = merged_df.groupby('Cluster')['X-Grade'].mean().reset_index()

# Step 5: Output the average X-Grade for each cluster
print(cluster_avg_grades)


In [ ]:
# Step 1: Calculate the standard deviation of X-Grade for each cluster
cluster_std_dev = merged_df.groupby('Cluster')['X-Grade'].std().reset_index()

# Step 2: Output the standard deviation for each cluster
print(cluster_std_dev)


In [ ]:
# Step 1: Sum the number of attempts for each student across problems
# Assuming that 'attempts_per_student_problem_pivot' contains the number of attempts for each student and problem
attempts_sum = merged_df.drop(columns=["SubjectID", "X-Grade", "Cluster"]).sum(axis=1)

# Step 2: Count the number of problems (columns) for each student
# Here we count the number of non-NaN values (problem attempts) for each student
num_problems = merged_df.drop(columns=["SubjectID", "X-Grade", "Cluster"]).notna().sum(axis=1)

# Step 3: Calculate the average number of attempts for each student
merged_df['average_problems'] = attempts_sum / num_problems

# Step 4: Plot the Grades vs Average Number of Attempts with Cluster-based coloring
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    merged_df['average_problems'],  # x-axis: Average Number of Attempts
    merged_df['X-Grade'],  # y-axis: X-Grade (Grades)
    c=merged_df['Cluster'],  # Color by Cluster
    cmap='viridis',  # You can change the colormap (e.g., 'viridis', 'plasma', 'coolwarm')
    alpha=0.7  # Transparency level for the dots
)

# Add colorbar to show which color corresponds to which cluster
plt.colorbar(scatter, label='Cluster')

# Step 5: Add labels and title to the plot
plt.xlabel("Average # of Problem Attempts")
plt.ylabel("Grade (X-Grade)")
plt.title("Scatter Plot: Grade vs Average # of Problem Attempts (Colored by Clusters)")

# Step 6: Show the plot
plt.show()



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Step 1: Merge the attempts dataset with subjects_table on SubjectID
merged_df = pd.merge(subjects_table, attempts_per_student_problem_pivot, on="SubjectID", how="left")
merged_df = merged_df[merged_df["X-Grade"] != 0]

# Step 2: Normalize the data (only the problem attempts)
scaler = StandardScaler()
normalized_data = scaler.fit_transform(merged_df.drop(columns=["SubjectID", "X-Grade"]))  # Drop non-relevant columns

# Step 3: Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
merged_df['Cluster'] = kmeans.fit_predict(normalized_data)

# Step 4: Apply PCA on the normalized data to reduce the problem attempts to 1 principal component
pca = PCA(n_components=1)
pc1 = pca.fit_transform(normalized_data)

# Step 5: Add the PCA result (PC1) back to the merged dataframe
merged_df['PC1'] = pc1

# Step 6: Plot the first principal component (PC1) against X-Grade, colored by cluster
plt.figure(figsize=(10, 6))
scatter = plt.scatter(merged_df['PC1'], merged_df['X-Grade'], c=merged_df['Cluster'], cmap='viridis', alpha=0.7)

# Step 7: Add color bar to show the cluster numbers
plt.colorbar(scatter, label='Cluster')

# Step 8: Add labels and title to the plot
plt.xlabel("Principal Component 1 (PC1) of Problem Attempts")
plt.ylabel("Grade (X-Grade)")
plt.title("Scatter Plot: Grade vs PCA of Problem Attempts (PC1), Colored by Clusters")

# Step 9: Show the plot
plt.show()

# Optionally, print the explained variance ratio to understand how much variance is explained by PC1
print(f"Explained variance ratio for PC1: {pca.explained_variance_ratio_[0]}")


In [ ]:
#if some errors are harder to shake /lead to lower grades
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

# Load datasets
main_df = pd.read_csv("MainTable.csv")
subject_df = pd.read_csv("Subject.csv")

# Step 1: Extract only the error message (remove line numbers)
def clean_compile_message(message):
    if pd.isna(message):  # Handle NaN values
        return None
    return re.sub(r'line \d+: ', '', message).strip()

def classify_error(error_message):
    if error_message is None:  # Handle None values
        return "Other"

    # Define keyword-based matching for classification
    error_categories = {
        "Syntax Error": [r"expected", r"missing", r"'.class' expected"],
        "Reference Error": [r"cannot find symbol", r"no suitable method", r"method .* cannot be applied"],
        "Type Error": [r"incompatible types", r"bad operand", r"required, but"],
        "Initialization Error": [r"might not have been initialized", r"already defined"],
        "Illegal Character": [r"illegal character", r"illegal '.'"],
        "Numeric Error": [r"integer number too large", r"array dimension missing"]
    }
    
    for category, patterns in error_categories.items():
        for pattern in patterns:
            if re.search(pattern, error_message, re.IGNORECASE):
                return category
    
    return "Other"



main_df['CompileMessageData'] = main_df['CompileMessageData'].apply(clean_compile_message)
main_df['CompileMessageData'] = main_df['CompileMessageData'].apply(classify_error)

# Step 2: Merge datasets on 'SubjectID'
merged_df = main_df.merge(subject_df, on="SubjectID", how="left")

# Step 3: Count occurrences of each error type per SubjectID
error_counts = merged_df.groupby(['SubjectID', 'CompileMessageData']).size().unstack(fill_value=0)

# Step 4: Merge error counts with X-Grade
final_df = error_counts.merge(subject_df, on="SubjectID", how="left")


# Step 5: Convert X-Grade to numeric
final_df['X-Grade'] = pd.to_numeric(final_df['X-Grade'], errors='coerce')


In [30]:

# -------------------- 🔥 Visualization 🔥 --------------------

# 1️⃣ Compute average grade for students who made each error vs. those who didn't
error_effects = {}
for error in error_counts.columns:
    students_with_error = final_df[final_df[error] > 0]
    students_without_error = final_df[final_df[error] == 0]

    avg_with_error = students_with_error["X-Grade"].mean()
    avg_without_error = students_without_error["X-Grade"].mean()

    error_effects[error] = avg_with_error - avg_without_error  # Difference in grades

# Convert to DataFrame
error_effects_df = pd.DataFrame(error_effects.items(), columns=["Error Type", "Grade Impact"])
error_effects_df = error_effects_df.sort_values("Grade Impact")


In [ ]:
# Set a threshold to filter out rare errors
min_occurrences = 10  # Adjust this based on dataset size

# Count how many students made each error
error_frequencies = error_counts.sum(axis=0)

# Merge with grade impact data and filter by occurrence count
filtered_errors = error_effects_df.merge(error_frequencies.rename("Frequency"), left_on="Error Type", right_index=True)
filtered_errors = filtered_errors[filtered_errors["Frequency"] >= min_occurrences]

# Sort by most negative grade impact and keep top 10
filtered_errors = filtered_errors.sort_values("Grade Impact").head(10)

# 🔥 Optimized Bar Plot
plt.figure(figsize=(12, 6))
sns.barplot(x=filtered_errors["Error Type"], y=filtered_errors["Grade Impact"], palette="coolwarm")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Grade Impact (Avg Grade Difference)")
plt.title("Most Harmful Errors on Student Grades")
plt.axhline(0, color='black', linewidth=1)  # Reference line at zero
plt.show()


In [ ]:
# Print all unique error types
unique_errors = error_counts.columns.tolist()
print("Unique Error Types:")
for error in unique_errors:
    print(error)

# Check how many different errors there are
print(f"\nTotal unique error types: {len(unique_errors)}")


In [ ]:

# 3️⃣ Scatter plot: Does making an error more often lower grades?
top_errors_list = error_effects_df["Error Type"].head(4)  # Pick top 4 most harmful errors

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, error in enumerate(top_errors_list):
    sns.regplot(x=final_df[error], y=final_df["X-Grade"], ax=axes[i], scatter_kws={'alpha':0.5})
    axes[i].set_title(f"X-Grade vs. '{error}' Occurrences")
    axes[i].set_xlabel(f"Count of '{error}'")
    axes[i].set_ylabel("X-Grade")

plt.tight_layout()
plt.show()